# Core Concepts: Pragmatic Functional Python

This notebook covers the foundational concepts of this blueprint. Each concept is a building block for writing robust, maintainable, and type-safe Python applications.

**Key Principle: Executable Documentation**
Every code example in this notebook is a mini-test. It ends with `assert` statements that prove its correctness. If you can run this notebook from top to bottom without any `AssertionError`, you can be confident that the examples work as described.

## 1. Data Validation with `Pydantic` as a "Boundary Guard"

**The Problem:** Your application's core logic should be a safe, predictable, and strongly-typed world. However, the outside world (APIs, databases, user input) is messy and unpredictable. How do you protect your core from malformed data?

**The Solution:** We use `Pydantic` models as a "Boundary Guard". At the edge of our application (e.g., in an API endpoint), we parse external data into a Pydantic model. If the data is valid, we get a clean, typed object to work with. If it's invalid, Pydantic raises a descriptive error, stopping bad data from ever entering our system.

### Best Practice: Using `typing.Annotated`

We prefer using `typing.Annotated` to declare validation rules. This is a modern Python feature (PEP 593) that cleanly separates the type (`str`) from the metadata (`Field(...)`). It makes the code more readable and interoperable with other tools that might also use annotations.

In [1]:
from typing import Annotated

from pydantic import BaseModel, Field, ValidationError


class User(BaseModel):
    """
    Represents a user, using Annotated for clean metadata.
    """

    id: int
    name: Annotated[str, Field(min_length=2, description="The user's name")]
    age: Annotated[int, Field(gt=0, le=120, description="Age in years")]


# --- Verification ---

# 1. Test the successful case
valid_data = {"id": 1, "name": "Alice", "age": 30}
user = User.model_validate(valid_data)

assert user.id == 1
assert user.name == "Alice"
assert user.age == 30

print("✅ Pydantic model created successfully from valid data.")

# 2. Test the validation failure case
invalid_data = {"id": 2, "name": "B", "age": 200}  # Name too short, age too high

raised_exception = None
try:
    User.model_validate(invalid_data)
except ValidationError as e:
    raised_exception = e

assert raised_exception is not None, "ValidationError was not raised for invalid data!"

error_messages = str(raised_exception)
assert "name" in error_messages
assert "String should have at least 2 characters" in error_messages
assert "age" in error_messages
assert "Input should be less than or equal to 120" in error_messages

print("✅ Pydantic correctly raised a ValidationError for invalid data.")

✅ Pydantic model created successfully from valid data.
✅ Pydantic correctly raised a ValidationError for invalid data.


## 2. Error Handling with `returns.Result`

**The Problem:** Traditional error handling with `try...except` blocks can make business logic hard to follow. It mixes the "happy path" with error-handling code and makes it unclear which functions can fail and which can't.

**The Solution:** We use the `Result` monad from the `returns` library, a pattern often called "Railway Oriented Programming".
- A function that can fail returns either `Success(value)` or `Failure(error)`.
- The return type `Result[SuccessType, FailureType]` makes it **explicit in the type signature** that the operation can fail.
- This forces the caller to handle the failure case, leading to more robust and predictable code.

In [2]:
from returns.result import Failure, Result, Success


def parse_user_id(raw_id: str) -> Result[int, str]:
    """Tries to parse a string into a positive integer ID."""
    if not raw_id.isdigit():
        return Failure(f"Invalid format: '{raw_id}' is not a digit.")

    user_id = int(raw_id)
    if user_id <= 0:
        return Failure(f"Invalid value: ID {user_id} must be positive.")

    return Success(user_id)


# --- Verification ---

# 1. Test the Success case
success_result = parse_user_id("123")
assert isinstance(success_result, Success)
assert success_result.unwrap() == 123
print("✅ `parse_user_id` returned Success for valid input.")

# 2. Test the Failure case (format)
failure_result_format = parse_user_id("abc")
assert isinstance(failure_result_format, Failure)
assert "Invalid format" in failure_result_format.failure()
print("✅ `parse_user_id` returned Failure for invalid format.")

# 3. Test the Failure case (value)
failure_result_value = parse_user_id("0")
assert isinstance(failure_result_value, Failure)
assert "must be positive" in failure_result_value.failure()
print("✅ `parse_user_id` returned Failure for invalid value.")

✅ `parse_user_id` returned Success for valid input.
✅ `parse_user_id` returned Failure for invalid format.
✅ `parse_user_id` returned Failure for invalid value.


## 3. Functional Pipelines with `returns.pipe`

**The Problem:** Chaining multiple data transformation steps can lead to deeply nested function calls that are hard to read from inside-out, or a series of difficult-to-track intermediate variables.

**The Solution:** We use `returns.pipe` to create a clean, readable, left-to-right data processing pipeline. It takes a starting value and passes it through a series of functions, where the output of one function becomes the input for the next.

In [3]:
from returns.pipeline import pipe


# Define a few simple, pure functions for our pipeline
def clean_text(text: str) -> str:
    return text.strip().lower()


def truncate_text(text: str) -> str:
    return text[:10]


def emphasize_text(text: str) -> str:
    return f'"{text}!"'


# The input data
raw_input = "  This is a Long String of Text   "

# Use pipe to compose the functions into a pipeline
processed_text = pipe(clean_text, truncate_text, emphasize_text)(raw_input)  # pyright: ignore

# --- Verification ---
print(f"Pipeline: '{raw_input}' to '{processed_text}'")
expected_output = '"this is a !"'
assert processed_text == expected_output
print("✅ Pipeline correctly working")

Pipeline: '  This is a Long String of Text   ' to '"this is a !"'
✅ Pipeline correctly working


## 4. Structured Logging with `Loguru`

**The Problem:** Python's built-in logging is powerful but requires a lot of boilerplate to set up. Getting useful, colored, and structured logs can be a chore.

**The Solution:** We use `Loguru` for a vastly improved developer experience. It provides sensible defaults, is easy to configure, and works out-of-the-box.

### Configuration via `.env`

This project configures Loguru based on environment variables defined in your `.env` file. You can control the log level and whether logs are written to a file without changing any code. See `.env.example` for details:

```dotenv
# .env.example
LOG_LEVEL="INFO"
LOG_TO_FILE="FALSE"
```

### Example Usage
The code below demonstrates basic logging. Since we can't easily assert on `stderr` output in a notebook, this cell is for demonstration. The `setup_logging` function is what you would call once at the start of your application.

In [4]:
import sys

from loguru import logger


# This is a simplified version of the project's logging setup
def setup_simple_logging(level="INFO"):
    logger.remove()
    logger.add(
        sys.stderr,
        level=level.upper(),
        format="<level>{level: <8}</level> | <cyan>{name}:{function}:{line}</cyan> - <level>{message}</level>",
        colorize=True,
    )


setup_simple_logging(level="DEBUG")

logger.debug("This is a debug message. Useful for developers.")
logger.info("Application is starting up...")
logger.success("A task was completed successfully.")
logger.warning("Something looks a bit strange, but it's not an error.")
logger.error("An error occurred! This needs attention.")

# No assert here, this is for visual inspection of the output above this cell.
print("\n✅ Loguru demonstrated various log levels.")

DEBUG    | __main__:<module>:19 - This is a debug message. Useful for developers.
INFO     | __main__:<module>:20 - Application is starting up...
SUCCESS  | __main__:<module>:21 - A task was completed successfully.
WARNING  | __main__:<module>:22 - Something looks a bit strange, but it's not an error.
ERROR    | __main__:<module>:23 - An error occurred! This needs attention.



✅ Loguru demonstrated various log levels.


### Focused Debugging: Filtering Logs

A powerful feature of Loguru is the ability to filter logs to focus on a specific part of your application. You don't configure this in the `.env` file, but directly in your logging setup code (e.g., `src/core/logging_config.py`) during a debugging session.

**Example: See only `DEBUG` messages from `my_module`**
```python
# Show only messages with level DEBUG from any module named 'my_module' or its children.
logger.add("debug.log", level="DEBUG", filter="my_module")
```

**Example: Ignore a noisy module**
```python
# Show all INFO logs, except for those coming from the noisy 'noisy_library'.
logger.add("filtered.log", level="INFO", filter=lambda record: "noisy_library" not in record["name"])
```